# Regression 02 : AirQuality

#### 머신러닝 코드 구조 <br>

![이미지](https://github.com/DA4BAM/dataset/blob/master/new_code.png?raw=true "code step1")



# [2] NA 처리 + Feature Engineering + Dummy Variable

In [1]:
#라이브러리들을 불러오자.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
data_path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/airquality_simple.csv'
data = pd.read_csv(data_path)

## 20.데이터 준비

### 21.변수 정리

### 22.NA 처리

* ffill로 채워봅시다.

In [3]:
# 다시 NA가 몇건 있는지 살펴봅시다.
data.isnull().sum()

Ozone      0
Solar.R    7
Wind       0
Temp       0
Month      0
Day        0
dtype: int64

In [4]:
# 시계열 데이터일 경우 쉬운 결정 : 이전 데이터로 채우기
data.fillna(method= 'ffill', inplace=1)

data.isnull().sum()

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41,190.0,7.4,67,5,1
1,36,118.0,8.0,72,5,2
2,12,149.0,12.6,74,5,3
3,18,313.0,11.5,62,5,4
4,19,313.0,14.3,56,5,5
...,...,...,...,...,...,...
148,30,193.0,6.9,70,9,26
149,23,145.0,13.2,77,9,27
150,14,191.0,14.3,75,9,28
151,18,131.0,8.0,76,9,29


* 이번에는 knn Imputation을 사용하여 추정해서 넣어 봅시다.  
    그러나 이 방법은 순서상 data split 이후에 진행합니다.

### 23.Feature Engineering

* 날짜 변수 만들기  
airquality 데이터는 1973년 5~9월까지 데이터 입니다.  
month, day 데이터를 날짜 형식으로 변환해 봅시다.

(중요!)날짜변수를 만드는 이유!  
* 날짜로 부터 추가 변수를 도출해 내기 위해서 날짜 변수를 만듭니다.  
* 만약 추가변수 도출이 끝나면, 날짜변수는 제거합니다.

In [ ]:
data.head()

In [5]:
# 날짜를 yyyymmdd로 만든 후 변환해봅시다.
data['Date'] = pd.to_datetime(1973*10000 + data.Month*100 + data.Day
                              ,format='%Y%m%d')

data.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Date
0,41,190.0,7.4,67,5,1,1973-05-01
1,36,118.0,8.0,72,5,2,1973-05-02
2,12,149.0,12.6,74,5,3,1973-05-03
3,18,313.0,11.5,62,5,4,1973-05-04
4,19,NaN,14.3,56,5,5,1973-05-05


In [ ]:
# 기간별 Ozone 차트를 그려봅시다.
data.plot(x='Date',y='Ozone')
plt.show()

* 날짜로 부터 날짜요소 추출하기

In [6]:
# 요일을 추가해 봅시다. 
data['WeekDay'] = data.Date.dt.dayofweek
data.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Date,WeekDay
0,41,190.0,7.4,67,5,1,1973-05-01,1
1,36,118.0,8.0,72,5,2,1973-05-02,2
2,12,149.0,12.6,74,5,3,1973-05-03,3
3,18,313.0,11.5,62,5,4,1973-05-04,4
4,19,NaN,14.3,56,5,5,1973-05-05,5


In [ ]:
# 아래 코드 끝에 . 을 찍으면 추출할 수 있는 날짜 요소를 확인할수 있습니다.
data.Date.dt.

In [7]:
# 날짜로 부터 변수 한 개 추가해 봅시다.

data['WeekNum'] = data.Date.dt.week
data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,Ozone,Solar.R,Wind,Temp,Month,Day,Date,WeekDay,WeekNum
0,41,190.0,7.4,67,5,1,1973-05-01,1,18
1,36,118.0,8.0,72,5,2,1973-05-02,2,18
2,12,149.0,12.6,74,5,3,1973-05-03,3,18
3,18,313.0,11.5,62,5,4,1973-05-04,4,18
4,19,NaN,14.3,56,5,5,1973-05-05,5,18


* 이전 데이터 붙이기.  
    - .shift()  
    - 예) 전날 오존농도, 전날 온도

In [8]:
# 전날 오존 농도를 추가합니다.
data['Ozone_lag1'] = data['Ozone'].shift() # default = 1
data.head(10)

,Ozone,Solar.R,Wind,Temp,Month,Day,Date,WeekDay,WeekNum,Ozone_lag1
0,41,190.0,7.4,67,5,1,1973-05-01,1,18,NaN
1,36,118.0,8.0,72,5,2,1973-05-02,2,18,41.0
2,12,149.0,12.6,74,5,3,1973-05-03,3,18,36.0
3,18,313.0,11.5,62,5,4,1973-05-04,4,18,12.0
4,19,NaN,14.3,56,5,5,1973-05-05,5,18,18.0
5,28,NaN,14.9,66,5,6,1973-05-06,6,18,19.0
6,23,299.0,8.6,65,5,7,1973-05-07,0,19,28.0
7,19,99.0,13.8,59,5,8,1973-05-08,1,19,23.0
8,8,19.0,20.1,61,5,9,1973-05-09,2,19,19.0
9,20,194.0,8.6,69,5,10,1973-05-10,3,19,8.0


다음의 feature들을 추가해 봅시다.

In [9]:
# 전날 온도, 전날 바람세기, 전날 solar.R
data['Temp_lag1'] = data['Temp'].shift()
data['Wind_lag1'] = data['Wind'].shift()
data['Solar.R_lag1'] = data['Solar.R'].shift()

In [10]:
# 전전날 오존농도 대비 전날 오존농도 증감
data['Ozone_diff'] = data.Ozone.shift(2) - data['Ozone_lag1'] 
data.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Date,WeekDay,WeekNum,Ozone_lag1,Temp_lag1,Wind_lag1,Solar.R_lag1,Ozone_diff
0,41,190.0,7.4,67,5,1,1973-05-01,1,18,NaN,NaN,NaN,NaN,NaN
1,36,118.0,8.0,72,5,2,1973-05-02,2,18,41.0,67.0,7.4,190.0,NaN
2,12,149.0,12.6,74,5,3,1973-05-03,3,18,36.0,72.0,8.0,118.0,5.0
3,18,313.0,11.5,62,5,4,1973-05-04,4,18,12.0,74.0,12.6,149.0,24.0
4,19,NaN,14.3,56,5,5,1973-05-05,5,18,18.0,62.0,11.5,313.0,-6.0


In [ ]:
data.head(10)

* 이동평균 변수 붙이기  
    - .rolling()  
    - 예) 7일 이동평균 오존농도


In [15]:
# 7일 이동평균 오존농도
data['Ozone_MA_7_lag1'] = data['Ozone'].rolling(7, min_periods=1).mean().shift()
data.head(10)

,Ozone,Solar.R,Wind,Temp,Month,Day,Date,WeekDay,WeekNum,Ozone_lag1,Temp_lag1,Wind_lag1,Solar.R_lag1,Ozone_diff,Ozone_MA_7_lag1,Temp_MA_3_lsg1
0,41,190.0,7.4,67,5,1,1973-05-01,1,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36,118.0,8.0,72,5,2,1973-05-02,2,18,41.0,67.0,7.4,190.0,NaN,41.000000,67.000000
2,12,149.0,12.6,74,5,3,1973-05-03,3,18,36.0,72.0,8.0,118.0,5.0,38.500000,69.500000
3,18,313.0,11.5,62,5,4,1973-05-04,4,18,12.0,74.0,12.6,149.0,24.0,29.666667,71.000000
4,19,NaN,14.3,56,5,5,1973-05-05,5,18,18.0,62.0,11.5,313.0,-6.0,26.750000,69.333333
5,28,NaN,14.9,66,5,6,1973-05-06,6,18,19.0,56.0,14.3,NaN,-1.0,25.200000,64.000000
6,23,299.0,8.6,65,5,7,1973-05-07,0,19,28.0,66.0,14.9,NaN,-9.0,25.666667,61.333333
7,19,99.0,13.8,59,5,8,1973-05-08,1,19,23.0,65.0,8.6,299.0,5.0,25.285714,62.333333
8,8,19.0,20.1,61,5,9,1973-05-09,2,19,19.0,59.0,13.8,99.0,4.0,22.142857,63.333333
9,20,194.0,8.6,69,5,10,1973-05-10,3,19,8.0,61.0,20.1,19.0,11.0,18.142857,61.666667


In [13]:
# 3일 이동평균 온도를 추가합시다.
data['Temp_MA_3_lsg1'] = data['Temp'].rolling(3, min_periods = 1).mean().shift()
data.head()


,Ozone,Solar.R,Wind,Temp,Month,Day,Date,WeekDay,WeekNum,Ozone_lag1,Temp_lag1,Wind_lag1,Solar.R_lag1,Ozone_diff,Ozone_MA_7_lag1,Temp_MA_3_lsg1
0,41,190.0,7.4,67,5,1,1973-05-01,1,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36,118.0,8.0,72,5,2,1973-05-02,2,18,41.0,67.0,7.4,190.0,NaN,41.000000,67.000000
2,12,149.0,12.6,74,5,3,1973-05-03,3,18,36.0,72.0,8.0,118.0,5.0,38.500000,69.500000
3,18,313.0,11.5,62,5,4,1973-05-04,4,18,12.0,74.0,12.6,149.0,24.0,29.666667,71.000000
4,19,NaN,14.3,56,5,5,1973-05-05,5,18,18.0,62.0,11.5,313.0,-6.0,26.750000,69.333333


In [ ]:
data.head(10)

### 24.Dummy Variable
    * 범주형 변수를 숫자로 만드는 방법
    * pd.get_dummies, pd.concat, (Pandas Dataframe).drop
    * 불필요한 칼럼들 제거


In [16]:
# dummy variable
data_wd = pd.get_dummies(data['WeekDay'], prefix = 'W', drop_first = 1)

In [17]:
data_wd.head()

,W_1,W_2,W_3,W_4,W_5,W_6
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,1,0,0,0
3,0,0,0,1,0,0
4,0,0,0,0,1,0


In [18]:
data2 = pd.concat([data, data_wd], axis=1)

In [19]:
data2.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Date,WeekDay,WeekNum,Ozone_lag1,Temp_lag1,Wind_lag1,Solar.R_lag1,Ozone_diff,Ozone_MA_7_lag1,Temp_MA_3_lsg1,W_1,W_2,W_3,W_4,W_5,W_6
0,41,190.0,7.4,67,5,1,1973-05-01,1,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0
1,36,118.0,8.0,72,5,2,1973-05-02,2,18,41.0,67.0,7.4,190.0,NaN,41.000000,67.000000,0,1,0,0,0,0
2,12,149.0,12.6,74,5,3,1973-05-03,3,18,36.0,72.0,8.0,118.0,5.0,38.500000,69.500000,0,0,1,0,0,0
3,18,313.0,11.5,62,5,4,1973-05-04,4,18,12.0,74.0,12.6,149.0,24.0,29.666667,71.000000,0,0,0,1,0,0
4,19,NaN,14.3,56,5,5,1973-05-05,5,18,18.0,62.0,11.5,313.0,-6.0,26.750000,69.333333,0,0,0,0,1,0


* 불필요한 변수를 제거합시다.

어떤 변수가 불필요한가요?

In [20]:
# 칼럼삭제
drop_x = ['Solar.R','Wind','Temp','Month','Day','Date','WeekDay','WeekNum']
data2.drop(drop_x, axis = 1, inplace = True)

In [21]:
data2.head()

,Ozone,Ozone_lag1,Temp_lag1,Wind_lag1,Solar.R_lag1,Ozone_diff,Ozone_MA_7_lag1,Temp_MA_3_lsg1,W_1,W_2,W_3,W_4,W_5,W_6
0,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0
1,36,41.0,67.0,7.4,190.0,NaN,41.000000,67.000000,0,1,0,0,0,0
2,12,36.0,72.0,8.0,118.0,5.0,38.500000,69.500000,0,0,1,0,0,0
3,18,12.0,74.0,12.6,149.0,24.0,29.666667,71.000000,0,0,0,1,0,0
4,19,18.0,62.0,11.5,313.0,-6.0,26.750000,69.333333,0,0,0,0,1,0


In [23]:
data2 = data2.fillna(method = 'bfill')
data2.head()

,Ozone,Ozone_lag1,Temp_lag1,Wind_lag1,Solar.R_lag1,Ozone_diff,Ozone_MA_7_lag1,Temp_MA_3_lsg1,W_1,W_2,W_3,W_4,W_5,W_6
0,41,41.0,67.0,7.4,190.0,5.0,41.000000,67.000000,1,0,0,0,0,0
1,36,41.0,67.0,7.4,190.0,5.0,41.000000,67.000000,0,1,0,0,0,0
2,12,36.0,72.0,8.0,118.0,5.0,38.500000,69.500000,0,0,1,0,0,0
3,18,12.0,74.0,12.6,149.0,24.0,29.666667,71.000000,0,0,0,1,0,0
4,19,18.0,62.0,11.5,313.0,-6.0,26.750000,69.333333,0,0,0,0,1,0


### 25.Data Split

데이터셋을 나눠봅시다.  
위 코드를 참조해서 직접 코드를 작성합시다. 


In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X = data2.iloc[:,1: ]
y = data2.iloc[:,0]

In [26]:
train_x, test_x, train_y, test_y = train_test_split(X,y, test_size = 0.3)

### 26.Scaling features




## 30.모델링
모델링을 다시 수행해 봅시다.

### 31.import

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error


### 32.모델선언

In [28]:
model = LinearRegression()


### 33.모델링(학습)

In [29]:
model.fit(train_x, train_y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [35]:
print(X.columns.to_list())
print(model.coef_)
print(model.intercept_)

['Ozone_lag1', 'Temp_lag1', 'Wind_lag1', 'Solar.R_lag1', 'Ozone_diff', 'Ozone_MA_7_lag1', 'Temp_MA_3_lsg1', 'W_1', 'W_2', 'W_3', 'W_4', 'W_5', 'W_6']
[ 0.48950104  0.72623944  0.28256251 -0.0278409   0.06552459 -0.01468689
  0.0865446   4.10413912 -4.28910841 -9.83641559 -7.7038053  -1.39506768
 -6.0314172 ]
-37.201625329106285


### 34.예측

In [33]:
test_pred = model.predict(test_x)

### 35.평가

In [34]:
# MSE, RMSE, MAE
print(mean_squared_error(test_y, test_pred))
print(mean_squared_error(test_y, test_pred, squared=False))
print(mean_absolute_error(test_y, test_pred))

856.3784695592808
29.263944873500577
20.104450150436787
